## Imports

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Input

In [2]:
gt = pd.read_csv("input/gt.csv")
gt

,x,y,w,h,c
0,10,0,100,100,1
1,200,300,95,200,1
2,1000,500,250,300,1
3,700,500,98,112,1
4,690,212,102,108,1


In [3]:
pred = pd.read_csv("input/pred.csv")
pred

,x,y,w,h,c,conf
0,710,530,83,92,1,0.4
1,653,250,79,94,1,0.6
2,205,310,100,196,1,0.7
3,1020,490,200,312,1,0.5
4,123,463,335,1023,1,0.8
5,463,230,1020,350,1,0.3
6,5,7,101,103,1,0.9


In [40]:
def get_segments(df):
    x_segments = [df['x'], df['x'] + df['w']]
    y_segments = [df['y'], df['y'] + df['h']]
    return np.array([x_segments, y_segments]).transpose(0, 2, 1)

In [76]:
def get_intersection(source, target):
    # Horizontal segments intersection
    x_intersection = get_segment_intersection(source[0], target[0])

    # Vertical segments intersection
    y_intersection = get_segment_intersection(source[1], target[1])

    return x_intersection * y_intersection

def get_segment_intersection(seg1, seg2):
    seg1_exp = np.expand_dims(seg1, axis=1).repeat(len(seg2), axis=1)
    seg2_exp = np.expand_dims(seg2, axis=0).repeat(len(seg1), axis=0)

    # Get the Cartesian product of input segments (each with each)
    pairs = np.array([seg1_exp, seg2_exp])
    pairs = pairs.transpose(1, 2, 0, 3)

    # Left and right limits of each pair
    l, r = pairs[..., 0], pairs[..., 1]

    return np.clip(np.min(r, axis=2) - np.max(l, axis=2), 0, None)

In [77]:
get_segments(pred)

array([[[ 710,  793],
        [ 653,  732],
        [ 205,  305],
        [1020, 1220],
        [ 123,  458],
        [ 463, 1483],
        [   5,  106]],

       [[ 530,  622],
        [ 250,  344],
        [ 310,  506],
        [ 490,  802],
        [ 463, 1486],
        [ 230,  580],
        [   7,  110]]], dtype=int64)

In [78]:
get_segments(gt)

array([[[  10,  110],
        [ 200,  295],
        [1000, 1250],
        [ 700,  798],
        [ 690,  792]],

       [[   0,  100],
        [ 300,  500],
        [ 500,  800],
        [ 500,  612],
        [ 212,  320]]], dtype=int64)

In [79]:
get_intersection(get_segments(pred), get_segments(gt))

array([[    0,     0,     0,  6806,     0],
       [    0,     0,     0,     0,  2940],
       [    0, 17100,     0,     0,     0],
       [    0,     0, 60000,     0,     0],
       [    0,  3515,     0,     0,     0],
       [    0,     0, 20000,  7840,  9180],
       [ 8928,     0,     0,     0,     0]], dtype=int64)